# What is JuMP?

JuMP is a _modeling language_ for optimization problems, writen in Julia. 

When solving an optimization problem, you should use a _solver_ :  a software implementation of an optimization algorithm. Solvers typically need the problem to be written in a very specific format, which can be time-consuming and confusing.

A modeling language (like JuMP) lets you code an optimization problem in a more natural way. It does the translation to the low-level solver format for you.

There are a number of modeling languages out there. Why JuMP?

* User-friendly
* Matches performance of competitors
* Solver-independent
* Easy to extend and take advantage of advanced features


# Installing JuMP

To install JuMP, just run

In [ ]:
#import Pkg
#Pkg.add("JuMP")

We already did this in the preassignment. To actually solve a problem, we will also need to install a solver package. There are 15+ options exposed in julia, each with support for different problem classes, different performance profiles, licensing requirements, etc. For the preassignment, we installed Gurobi, a best-of-breed linear/integer programming solver with a generous academic license.

In [ ]:
#Pkg.add("Gurobi")

# A first example
Let's see how we translate a simple, 2 variable LP to JuMP code.

$$
\begin{align*}
\max_{x,y} \quad& x + 2y \\
\text{s.t.}\quad& x + y \leq 1 \\
& x, y \geq 0.
\end{align*}
$$

First, we load the JuMP and Gurobi libraries.

In [4]:
using JuMP, Gurobi

Next, we construct a model object. This is a container for everything in our optimization problem: variables, constraints, solver options, etc.
Note that when we specify the solver we can also pass additional arguments (depending on the solver we choose). 
If you're experimenting with an optimization model, it's generally a good idea to set a short time limit for the solver calculations. This will (hopefully) force the solver to stop running if you accidentally pass it a very large model that it can't solve. 

In [5]:
model = Model(with_optimizer(Gurobi.Optimizer, TimeLimit=60))

Academic license - for non-commercial use only


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

Next, we define the two decision variables in our optimization problem. We will use the ``@variable`` macro (a fancy function, essentially). The first argument is the model object to attach the variable to, and the second specifies the variable name and any bounds.

In [3]:
@variable(model, x>=0)
@variable(model, y>=0)

y

In [4]:
model

A JuMP Model
Feasibility problem with:
Variables: 2
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: x, y

We now add the single constraint of our problem using the ``@constraint`` macro. We write it algebraically, exactly as we see it above.

In [5]:
@constraint(model, x+y<=1)
model

A JuMP Model
Feasibility problem with:
Variables: 2
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: x, y

We specify the objective function with the ``@objective`` macro.

In [6]:
@objective(model, Max, x + 2y)

x + 2 y

In [7]:
model

A JuMP Model
Maximization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: x, y

To solve the optimization problem, call the ``optimize!`` function.

In [8]:
optimize!(model)

Academic license - for non-commercial use only
Optimize a model with 1 rows, 2 columns and 2 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  2.000000000e+00


In [9]:
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

We can now inspect the solution values and optimal cost.

In [11]:
JuMP.objective_value(model)

2.0

In [12]:
JuMP.value(x)

0.0

In [13]:
JuMP.value(y)

1.0

# Exercise 1

Code and solve the following optimization problem:

$$
\begin{align*}
\min_{x,y} \quad& 3x - y \\
\text{s.t.}\quad& x + 2y \geq 1 \\
& x \geq 0 \\
& 0 \leq y \leq 1.
\end{align*}
$$

In [7]:
model = Model(with_optimizer(Gurobi.Optimizer, TimeLimit=60))
@variable(model, x>=0)
@variable(model, 1>=y>=0)
@constraint(model, x+2y>=1)
@objective(model, Min, 3x - y)
model
optimize!(model)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Optimize a model with 1 rows, 2 columns and 2 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective -1.000000000e+00


In [15]:
JuMP.objective_value(model)


-1.0

In [8]:
JuMP.value(x)

0.0

In [17]:
JuMP.value(y)


1.0

# Exercise 2

Take the problem from exercise 1 and make two copies below.
* In the first copy, add a new constraint to make the problem **infeasible** (i.e., there are no values of x and y that satisfy all the constraints)
* In the second copy, change the constraints or the objective function to make the problem unbounded (i.e., the optimal solution is infinite)

Solve both versions of the problem and look at the ``termination_status`` to see if you have succeeded.

In [20]:
model = Model(with_optimizer(Gurobi.Optimizer, TimeLimit=60))
@variable(model, x>=0)
@variable(model, 1>=y>=0)
@constraint(model, 0> x+2y )
@objective(model, Min, 3x - y)
optimize!(model)

Academic license - for non-commercial use only


LoadError: In `@constraint(model, 0 > x + 2y)`: Unrecognized sense >

In [21]:
model = Model(with_optimizer(Gurobi.Optimizer, TimeLimit=60))
@variable(model, x>=0)
@variable(model, y>=0)
@constraint(model, x+2y>=1)
@objective(model, Min, 3x - y)
optimize!(model)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Optimize a model with 1 rows, 2 columns and 2 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.0000000e+30   0.000000e+00   1.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Unbounded model
